вариант 6 Морозов Сергей

In [1]:
import numpy as np
import scipy

1Создать произвольную верхнетреугольную матрицу А 5 по-рядка   (не   унитреугольную),   вектор  B  произвольный.   Решить   си-стему AX =  B

In [2]:
# Создание произвольной верхнетреугольной матрицы A 5x5
A = np.array([[3, 1, 2, 4, 2],
              [0, 2, 0, 1, 5],
              [0, 0, 4, 0, 3],
              [0, 0, 0, 2, 6],
              [0, 0, 0, 0, 1]])

# Создание произвольного вектора B
B = np.array([15, 8, 11, 12, 3])

# Решение системы AX = B с помощью обратной подстановки
n = len(B)
X = np.zeros(n)

for i in range(n - 1, -1, -1):
    X[i] = (B[i] - np.dot(A[i, i+1:], X[i+1:])) / A[i, i]

print("Решение X:")
print(X)

Решение X:
[ 7.33333333 -2.          0.5        -3.          3.        ]


2Решить систему, используя LU   разложение
4.3x1−12.1x2+23.2x3−14.1x4=15.5
2.4x1−4.4x2+3.5x3+5.5x4=2.5
5.4x1+8.3x2−7.4x3−12.7x4=8.6
6.3x1−7.6x2+1.34x3+3.7x4=12.1

In [6]:
# Функция для вычисления LU-разложения
def lu_decomposition(matrix):
    n = matrix.shape[0]
    L = np.eye(n)  # Создаем начальную матрицу L как единичную
    U = matrix.copy()  # Создаем копию исходной матрицы для U

    for key in range(n):
        for i in range(key+1, n):
            if U[key, key] == 0:
                raise ValueError("LU-разложение невозможно: ведущий элемент равен нулю.")
            factor = U[i, key] / U[key, key]
            L[i, key] = factor
            U[i, key:] -= factor * U[key, key:]

    return L, U

# Матрица коэффициентов системы
A = np.array([[4.3, -12.1, 23.2, -14.1],
              [2.4, -4.4, 3.5, 5.5],
              [5.4, 8.3, -7.4, -12.7],
              [6.3, -7.6, 1.34, 3.7]])

# Вектор правой части
B = np.array([15.5, 2.5, 8.6, 12.1])

# Выполним LU-разложение матрицы A
L, U = lu_decomposition(A)

U_inverse = np.linalg.inv(U)
L_inverse = np.linalg.inv(L)
Y = L_inverse.dot(B)
X_lu = U_inverse.dot(Y)
print("Решение X:")
print(X_lu)

Решение X:
[ 1.17377014 -0.97122206 -0.40615705 -0.57615922]


3Решить   систему  из   пункта   2   с   помощью  QR  разложенияматрицы А. QR разложение найти методом Гивенса. Проверить по-лученное решение непосредственной подстановкой в исходную си-стему, а также методом np.solve.

In [10]:
matrix_A_float = A.astype(float) # меняем тип с int на float

# метод Гивенса (вращений)
def givens(matrix):
    (r, c) = np.shape(matrix)
    Q = np.identity(r)
    R = np.copy(matrix)
    (rows, cols) = np.tril_indices(r, -1, c)

    for (row, col) in zip(rows, cols):
        if R[row, col] != 0:
            r_ = np.hypot(R[col, col], R[row, col])
            c = R[col, col]/r_
            s = -R[row, col]/r_
            G = np.identity(r)
            G[[col, row], [col, row]] = c
            G[row, col] = s
            G[col, row] = -s
            R = np.dot(G, R)
            Q = np.dot(Q, G.T)

    return Q, R


# находим QR разложение
Q, R= givens(matrix_A_float)
R_inverse = np.linalg.inv(R)
Q_inverse = np.linalg.inv(Q)
Y = Q_inverse.dot(B)#  Y = (Q^-1)*B
x = R_inverse.dot(Y)# X = (R^-1)*Y
print("Ответ: "+str(x))
check = np.allclose(np.dot(A, x), B)
if check:
    print("Проверка непосредственной подстановкой: Решение верно.")
else:
    print("Проверка непосредственной подстановкой: Решение неверно.")
x_check = np.linalg.solve(A, B)
check2 = np.allclose(x, x_check)
if check2:
    print("Проверка с использованием np.linalg.solve: Решение верно.")
else:
    print("Проверка с использованием np.linalg.solve: Решение неверно.")

Ответ: [ 1.17377014 -0.97122206 -0.40615705 -0.57615922]
Проверка непосредственной подстановкой: Решение верно.
Проверка с использованием np.linalg.solve: Решение верно.


4Решить систему методом простых итераций с точностью до10−3.   Проверить   выполнение   достаточного   условия   сходимости.Если условие не выполняется, в программе выполнить эквивалент-ные   преобразования   системы,   после   этого   привести   к   удобномудля   итераций   виду.   Оформить  итерации   в   виде   таблицы   (можнопользоваться модулями PrettyTable или Pandas)

3.2x1−2.5x2+3.7x3=6.5
0.5x1+0.34x2+1.7x3=−0.24
1.6x1+2.3x2−1.5x3=4.3


In [21]:
import numpy as np
import pandas as pd

def simple_iteration():
    matrix_1= np.array([[3.2, -2.5, 3.7], [0.5, 0.34, 1.7], [1.6, 2.3, -1.5]])

    # Правая часть уравнений (вектор B)
    vector_1 = np.array([6.5, -0.24, 4.3])

    # Создаем новую матрицу A' и вектор B', чтобы учесть правую часть
    matrix = matrix_1.copy()
    vector = vector_1.copy()

    # Проходим по каждой строке и увеличиваем главный диагональный элемент A_prime[i, i] так, чтобы он был больше суммы остальных элементов в строке
    for i in range(A.shape[0]):
        diagonal_element = abs(matrix[i, i])
        off_diagonal_sum = sum(abs(matrix[i, :])) - diagonal_element
        while diagonal_element <= off_diagonal_sum:
            diagonal_element += 1.0  # Увеличиваем главный диагональный элемент на 1
        matrix[i, i] = diagonal_element
        solutions = np.zeros((3, 1))
        vector_c = np.zeros((3, 1))
        tmp_matrix = np.zeros((3, 3))
    
    for i in range(3):
        for j in range(3):
            if i != j:
                tmp_matrix[i, j] = -matrix[i, j] / matrix[i, i]
            else:
                tmp_matrix[i, j] = 0
    
    for i in range(3):
        vector_c[i] = vector[i] / matrix[i, i]
    
    tmp_solutions = np.dot(tmp_matrix, solutions) + vector_c
    accuracy = 0.001
    df = pd.DataFrame({"x_1": [], "x_2": [], "x_3": [], "norma": []})
    
    while np.linalg.norm(tmp_solutions - solutions, np.inf) > accuracy:
        solutions = tmp_solutions
        tmp_solutions = np.dot(tmp_matrix, solutions) + vector_c
        df = df._append({"x_1": solutions[0][0], "x_2": solutions[1][0], "x_3": solutions[2][0],
                         "norma": np.linalg.norm(tmp_solutions - solutions, np.inf)}, ignore_index=True)
    
    print("Таблица с итерациями")
    print(df)
    
    print('Матрица A')
    print(matrix_1)
    print('Вектор ответов B')
    print(vector_1)
    print('Решения X')
    print(solutions)
  

simple_iteration()


Таблица с итерациями
         x_1       x_2       x_3     norma
0   0.902778 -0.102564  0.955556  0.887108
1   0.376116 -0.989672  0.686990  0.640668
2   0.206105 -0.682026  1.327658  0.429116
3  -0.016305 -1.111142  1.230865  0.298405
4  -0.115563 -0.993298  1.529270  0.195581
5  -0.227992 -1.188879  1.504330  0.139939
6  -0.283086 -1.146737  1.644269  0.089893
7  -0.340366 -1.236630  1.642319  0.066311
8  -0.370577 -1.222974  1.708630  0.041720
9  -0.399912 -1.264693  1.712392  0.031754
10 -0.416331 -1.261158  1.744145  0.019561
11 -0.431421 -1.280719  1.748176  0.015363
12 -0.440285 -1.280423  1.763539  0.009267
13 -0.448077 -1.289690  1.766539  0.007507
14 -0.452836 -1.290205  1.774047  0.004437
15 -0.456873 -1.294641  1.776002  0.003703
16 -0.459418 -1.295199  1.779705  0.002146
17 -0.461515 -1.297346  1.780895  0.001842
18 -0.462872 -1.297762  1.782738  0.001092
19 -0.463963 -1.298811  1.783433  0.000924
Матрица A
[[ 3.2  -2.5   3.7 ]
 [ 0.5   0.34  1.7 ]
 [ 1.6   2.3  -1.5 ]]
Ве

Найти псевдорешение системы:

3.2x1−2.5x2+3.7x3=6.5

0.5x1+0.34x2+1.7x3=−0.24

1.6x1+2.3x2−1.5x3=4.3

3.6x1+1.8x2−4.7x3=3.8

In [5]:
# Матрица коэффициентов A
A = np.array([[3.2, -2.5, 3.7],
              [0.5, 0.34, 1.7],
              [1.6, 2.3, -1.5],
              [3.6, 1.8, -4.7]])

# Вектор правой части b
b = np.array([6.5, -0.24, 4.3, 3.8])

# Вектор правой части b
b = np.array([6.5, -0.24, 4.3, 3.8])
matrix_A_5_transp = A.transpose()
tmp_1 = np.dot(matrix_A_5_transp, A)# A_transp * A 
tmp_2 = np.linalg.inv(tmp_1)# ( A_transp * A )^-1
tmp_3 = np.dot(np.dot(tmp_2, matrix_A_5_transp), B)# ( A_transp * A )^-1 * A_transp * B
print("\nПсевдорешение:\n", tmp_3)


Псевдорешение:
 [4.31936277 0.85634867 0.93090771]
